In [3]:
import orbit
from orbit.models import DLT
from orbit.diagnostics.plot import plot_predicted_data, plot_predicted_components
from orbit.diagnostics.backtest import BackTester
from orbit.diagnostics.metrics import mae, mse, smape, wmape

import pandas as pd
import pandahouse as ph
import numpy as np
import os

import seaborn as sns
import matplotlib.pyplot as plt
import arviz as az

sns.set(rc={'figure.figsize':(16,6)}, style="whitegrid")

In [4]:
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'
}

1. Выберите основную метрику, которую вы планируете прогнозировать. Обоснуйте, почему именно она. Какое временное разрешение вы возьмёте? Будут ли какие-то дополнительные регрессоры, которые вы включите в модель?

Основная метрика - число активных пользователей в день (Daily Active Users, DAU). Число активных пользователей напрямую связано с нагрузкой на серверы и инфраструктуру приложения. DAU чувствительна к изменениям в поведении пользователей, что позволяет своевременно выявлять и реагировать на проблемы с производительностью. DAU является стандартной метрикой в аналитике, что упрощает сравнение с историческими данными и метриками других компаний.

Регрессоры: </br>
День недели: Активность пользователей часто зависит от дня недели (больше активных пользователей в выходные или будние дни).</br>
Праздничные дни: Включение данных о праздниках может помочь учесть резкие изменения в активности пользователей.

2. Постройте модель и провалидируйте её. Хватает ли у нас данных для бэктестинга с текущей задачей? Если нет, то определите, для какого горизонта прогнозирования у нас хватает данных.

In [ ]:
q = """
SELECT 
    toDate(time) as day,
    action,
    user_id,
    post_id
FROM 
    simulator_20240620.feed_actions 
WHERE toDate(time) >= '2024-04-14' and toDate(time) <= '2024-07-04'
ORDER BY toDate(time)
"""
df = ph.read_clickhouse(q, connection=connection)

In [ ]:
df.head()

In [ ]:
act_df = df.set_index("day").resample("D").agg({"user_id": "nunique", 
                                              "action":"count"}).rename({"user_id":"DAU", 
                                                                         "action":"activity"}, axis=1).reset_index()

In [ ]:
act_df.head()

In [ ]:
query = '''
SELECT 
    toStartOfDay(time) day, 
    COUNT(receiver_id) actions_mes
FROM simulator_20240620.message_actions 
WHERE toDate(time) >= '2024-04-14' and toDate(time) <= '2024-07-04'
GROUP BY day
ORDER BY day
'''

mes_activity = ph.read_clickhouse(query=query, connection=connection)
mes_activity.head()

In [ ]:
act_df = act_df.merge(mes_activity)
act_df.head()

In [ ]:
act_df["total_activity"] = act_df["activity"] + act_df["actions_mes"]
act_df.head()

In [ ]:
sns.lineplot(data=act_df, x="day", y="DAU")

In [ ]:
sns.set(rc={'figure.figsize':(16,6)}, style="whitegrid")

In [ ]:
sns.lineplot(data=act_df, x="day", y="activity")

In [ ]:
sns.lineplot(data=act_df, x="day", y="actions_mes")